In [8]:
import os
import re
import time
import torch
import torch.nn as nn
import torch.autograd as autograd
import torch.optim as optim
import numpy as np
import torch.nn.functional as F
from torch.nn.utils.rnn import pack_padded_sequence
from torch.nn.utils.rnn import pad_packed_sequence
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

torch.manual_seed(1)

In [43]:
#just load the data
import preprocess
import utils.load_acc

stiffness_o, mass_o = preprocess.task()

acc = utils.load_acc.task()


(101, 153)


In [44]:
#extra process
from sklearn.preprocessing import StandardScaler

print("origin:")
print(stiffness_o)
print(mass_o)

ss_s, ss_m = StandardScaler(), StandardScaler()
ss_s.fit(stiffness_o);ss_m.fit(mass_o)

stiffness, mass = ss_s.transform(stiffness_o), ss_m.transform(mass_o)

print("after: ")
print(stiffness, mass)



origin:
(array([1.08796486e+69, 1.08808383e+69, 1.08908017e+69, 1.08885479e+69,
       1.08931804e+69, 1.08768576e+69, 1.08868542e+69, 1.08900052e+69,
       1.08920405e+69, 1.08833844e+69, 1.08931116e+69, 1.08477810e+69,
       1.08899540e+69, 1.08901423e+69, 1.08931106e+69, 1.08882217e+69,
       1.08874177e+69, 1.08911277e+69, 1.08856633e+69, 1.08930794e+69,
       1.08911155e+69, 1.08675478e+69, 1.08931483e+69, 1.08868772e+69,
       1.08920844e+69, 1.08918387e+69, 1.08928732e+69, 1.08931747e+69,
       1.08714420e+69, 1.08644231e+69, 1.08928509e+69, 1.08633191e+69,
       1.08819950e+69, 1.08655127e+69, 1.08804704e+69, 1.08569494e+69,
       1.08744433e+69, 1.08603763e+69, 1.08834232e+69, 1.08921891e+69,
       1.08758280e+69, 1.08907671e+69, 1.08851018e+69, 1.08926884e+69,
       1.08931794e+69, 1.08838769e+69, 1.08853736e+69, 1.08932341e+69,
       1.08363488e+69, 1.08899908e+69, 1.08884836e+69, 1.08910354e+69,
       1.08924674e+69, 1.08925671e+69, 1.08629646e+69, 1.08895196e+6

In [71]:
# trans to torch tensor
print(type(stiffness[0]))
print(len(mass[0]))

s = np.array([stiffness[0]])

print(s.shape)
for i in range(1, 3):
    s= np.concatenate((s, np.array([stiffness[i]])), axis=0)

for i in range(3):
    s = np.concatenate((s, np.array([mass[i]])), axis=0)

print(s.shape)

print(acc.shape)

src = torch.tensor(s, dtype=torch.float64)

tgt = torch.tensor(acc, dtype=torch.float64)

src = torch.unsqueeze(src, dim=1)

tgt = torch.unsqueeze(tgt, dim=1)

print(src.shape, tgt.shape)

#dataloader

print(src.shape)
print(tgt.shape)

for i in range(4):
    src = torch.cat((src, src), 1)

for i in range(4):
    tgt = torch.cat((tgt, tgt), 1)

print(src.shape)
print(tgt.shape)

print(tgt)




<class 'numpy.ndarray'>
153
(1, 153)
(6, 153)
(101, 153)
torch.Size([6, 1, 153]) torch.Size([101, 1, 153])


In [76]:
#test model
def pretty_time(time, degree=1):
    return str(int(time // 60)) + "m" + str(round(time % 60, degree)) if time > 60 else round(time, degree)


from net.model import FesNet

net = FesNet(element_dim=153)

#net = ESIM(embedding_dim=100, vocab_dim=vocab_size, hidden_size=20, class_num=n_classes, embedding=None, pad_idx=pad_idx)

criterion =  nn.MSELoss()

optimizer = optim.Adam(net.parameters(), lr=0.1)

start_time = time.time()

loss_list = []

print(src.shape, tgt.shape)

src, tgt = src.float(), tgt.float()

for i in range(100):
    total_loss = 0.

    probs = net(src, tgt)

    net.zero_grad()
  
    loss = criterion(probs.reshape(-1), tgt.reshape(-1))

    loss.backward()
    torch.nn.utils.clip_grad_norm_(net.parameters(), 0.001)
    optimizer.step()

    total_loss += loss.item()
    if (i+1) % 10 == 0:
        time_past = time.time() - start_time
        cur_loss = total_loss
        loss_list.append(cur_loss)
        total_loss = 0
        print("epoch: ", i, " time: ", pretty_time(time_past), "loss: ", cur_loss)
    if (i+1) % 20 == 0:
        pass
        #print(probs)
        #print(tgt)

probs = net(src, tgt)

print(probs)

plt.plot(loss_list)

plt.show()


torch.Size([6, 256, 153]) torch.Size([101, 256, 153])
epoch:  9  time:  1m1.9 loss:  157769.515625
epoch:  19  time:  1m36.9 loss:  157174.75


KeyboardInterrupt: 

In [74]:
print(tgt)

tensor([[[ 0.0000e+00,  0.0000e+00,  0.0000e+00,  ...,  0.0000e+00,
           0.0000e+00,  0.0000e+00],
         [ 0.0000e+00,  0.0000e+00,  0.0000e+00,  ...,  0.0000e+00,
           0.0000e+00,  0.0000e+00],
         [ 0.0000e+00,  0.0000e+00,  0.0000e+00,  ...,  0.0000e+00,
           0.0000e+00,  0.0000e+00],
         ...,
         [ 0.0000e+00,  0.0000e+00,  0.0000e+00,  ...,  0.0000e+00,
           0.0000e+00,  0.0000e+00],
         [ 0.0000e+00,  0.0000e+00,  0.0000e+00,  ...,  0.0000e+00,
           0.0000e+00,  0.0000e+00],
         [ 0.0000e+00,  0.0000e+00,  0.0000e+00,  ...,  0.0000e+00,
           0.0000e+00,  0.0000e+00]],

        [[ 0.0000e+00,  8.1540e+00,  3.5931e+01,  ..., -6.8008e-02,
          -6.7872e-02, -6.7872e-02],
         [ 0.0000e+00,  8.1540e+00,  3.5931e+01,  ..., -6.8008e-02,
          -6.7872e-02, -6.7872e-02],
         [ 0.0000e+00,  8.1540e+00,  3.5931e+01,  ..., -6.8008e-02,
          -6.7872e-02, -6.7872e-02],
         ...,
         [ 0.0000e+00,  8